#### Importações

In [38]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from pycaret.classification import ClassificationExperiment

In [39]:
# from google.colab import drive

# drive.mount('/content/drive')
# df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/censo_cultural.xlsx')

In [40]:
df = pd.read_excel('/mnt/g/Meu Drive/Colab Notebooks/censo_cultural.xlsx')

#### Functions

##### Renomear colunas

In [41]:
def rename_columns()->None:
    df.rename(columns={
    '2) Qual o seu curso?': 'curso',
    '3) Qual o Ano que está cursando? ': 'serie',
    '5) Você é um aluno(a)...  ': 'tipo_residente',
    '6) Quais dessas ATIVIDADES CULTURAIS você participa ou já participou?': 'atv_culturais',
    '7) Quais dessas HABILIDADES MUSICAIS você possui?': 'hab_musicais',
    '8) Quais desses ESPORTES você pratica?': 'esportes',
    '9) Quais desses JOGOS você pratica?': 'jogos',
    '10) Quais dessas LUTAS você pratica?': 'lutas',
    '11) Em relação às 5 perguntas anteriores, quais dessas atividades você se considera bom e capaz de ensinar aos seus colegas?': 'melhores_atividades',
    '12) Quais dessas ATIVIDADES CULTURAIS você gostaria de participar?': 'atv_culturais_a_fazer',
    '13) Quais dessas HABILIDADES MUSICAIS você gostaria de possuir?': 'hab_musicais_desejadas',
    '14) Quais desses ESPORTES você gostaria de praticar?': 'esportes_desejados',
    '15) Quais desses JOGOS você gostaria de praticar?': 'jogos_desejados',
    '16) Quais dessas LUTAS você gostaria de praticar?': 'lutas_desejadas',
    '17) Em quais HORÁRIOS você teria disponibilidades para exercer Atividades Culturais?': 'horarios_disponiveis',
    '18) Em quais LOCAIS você teria disponibilidades para exercer Atividades Culturais ': 'locais_disponiveis',
    '19) Em relação às ESTRUTURAS EXISTENTES, o que você gostaria que melhorasse em nossa instituição?': 'melhoria_estrutura',
    '20) Em relação às NOVAS ESTRUTURAS, o que você desejaria que houvesse em nossa instituição?': 'novas_estruturas',
    '21) Em relação à ALIMENTAÇÃO, o que você desejaria que fosse melhorado em nossa instituição?': 'melhoria_alimentacao',
    '22) Em relação ao TRANSPORTE, o que você desejaria que fosse melhorado?': 'transporte',
    '23) Quais desse PROJETOS ESPECIAIS, você teria interesse em participar?': 'projetos_a_participar'
}, inplace=True)

##### Tratamento de strings

In [42]:
def encoder(series_names: list)->None:
   for serie_name in series_names:
      enc = LabelEncoder()
      df[serie_name] = enc.fit_transform(df[serie_name])

In [43]:
import unidecode as uni

def remove_accent(name):
    return uni.unidecode(str(name))

##### Dummies

In [44]:
def insert_dummies(df: pd.DataFrame, dummies: pd.DataFrame, insert_position: int) -> None:
    df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
    df.drop('aux', axis=1, inplace=True)

#### Pré-processamento dos dados

In [45]:
df.drop(columns=['Carimbo de data/hora', '1) Nome Completo: ', '4) Qual a sua Turma? '], inplace=True)

In [46]:
rename_columns()

In [47]:
for column in df.columns:
    df[column] = df[column].str.lower()
    df[column] = df[column].apply(remove_accent)

In [48]:
encoder([
    'serie',
    'tipo_residente',
])

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   curso                   611 non-null    object
 1   serie                   611 non-null    int64 
 2   tipo_residente          611 non-null    int64 
 3   atv_culturais           611 non-null    object
 4   hab_musicais            611 non-null    object
 5   esportes                611 non-null    object
 6   jogos                   611 non-null    object
 7   lutas                   611 non-null    object
 8   melhores_atividades     611 non-null    object
 9   atv_culturais_a_fazer   611 non-null    object
 10  hab_musicais_desejadas  611 non-null    object
 11  esportes_desejados      611 non-null    object
 12  jogos_desejados         611 non-null    object
 13  lutas_desejadas         611 non-null    object
 14  horarios_disponiveis    611 non-null    object
 15  locais

#### Pré processamento - Valores unicos

##### atv_culturais

In [50]:
df['aux'] = df['atv_culturais'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('artes marciais', 'luta')
            .replace('capoeira', 'luta')
            .replace('karate', 'luta')
            .replace('canto', 'musica')
            .replace('outro: musica', 'musica')
            .replace('praticava futsal no ginasio da vila sao joao', 'futsal')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('atv_cultural_')

insert_position = df.columns.get_loc('atv_culturais') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

##### hab_musicais

In [51]:
df['aux'] = df['hab_musicais'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('hab_musical_')

insert_position = df.columns.get_loc('hab_musicais') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

##### esportes

In [52]:
df['aux'] = df['esportes'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('caminhada/corridaa', 'caminhada/corrida')
            .replace('futbol de campo', 'futebol de campo')
            .replace('futbol de salao', 'futebol de salao')
            .replace('handbol', 'handebol')
            .replace('musculacao', 'academia')
            .replace('karate', 'artes marciais')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('esporte_')

insert_position = df.columns.get_loc('esportes') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

##### jogos

In [53]:
df['aux'] = df['jogos'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('assim como o de mesa', 'rpg')
            .replace('filho da puta', 'nenhuma opcao')
            .replace('league of legends', 'jogos eletronicos')
            .replace('overwatch', 'jogos eletronicos')
            .replace("(toto)", '')
            .replace('pebolin', 'pebolin/toto')
            .replace('por exemplo o d&d', 'rpg')
            .replace('rpg variados', 'rpg')
            .replace('xadres', 'xadrez')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('jogo_')

insert_position = df.columns.get_loc('jogos') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

##### lutas

In [54]:
df['aux'] = df['lutas'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('ja pratiquei karate', 'karate')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('luta_')

insert_position = df.columns.get_loc('lutas') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

##### melhores_atividades <div style="color: #ce1b0c"> VALIDAR NECESSIDADE </div>

In [55]:
# df['aux'] = df['melhores_atividades'].apply(
#     lambda column: [
#         item.strip() for item in column
#             .replace(' / ', '/')
#             .replace('/ ', '/')
#             .replace(' /', '/')
#             .replace('ja pratiquei karate', 'karate')
#             .split(',')
#     ]
# )

# dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('melhores_atividades_')

# insert_position = df.columns.get_loc('melhores_atividades') + 1

# df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
# df.drop('aux', axis=1, inplace=True)

##### atv_culturais_a_fazer

In [56]:
df['aux'] = df['atv_culturais_a_fazer'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('aula de bateria', 'aprender algum instrumento')
            .replace('tocar teclado ou outros instrumentos.', 'aprender algum instrumento')
            .replace('(marquei poesia, mas sou boa em leitura, nao em escrever)', '')
            .replace('outras opcoes (especifique) ou observacoes', '')
            .replace('karate', 'luta')
            .replace('jiu-jitsu', 'luta')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('atv_cultural_a_fazer_')

insert_position = df.columns.get_loc('atv_culturais_a_fazer') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

##### hab_musicais_desejadas

In [57]:
df['aux'] = df['hab_musicais_desejadas'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('percussao', '')
            .replace('percussa', '')
            .replace('bateria/', 'bateria/percussao')
            .replace('violao/guitarra/demais instrumentos de corda', 'canto, instrumentos de corda')
            .replace('canto/violao/guitarra/demais instrumentos de corda', 'canto/canto, instrumentos de corda')
            .replace('canto/canto', 'canto')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('hab_musicais_desejadas_')

insert_position = df.columns.get_loc('hab_musicais_desejadas') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

##### esportes_desejados

In [58]:
df['aux'] = df['esportes_desejados'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('futbol de campo', 'futebol de campo')
            .replace('futebol de campo/basquete/handebol/pingue pongue/peteca/futmesa', 'futebol de campo, basquete, handebol, pingue pongue, peteca, futmesa')
            .replace('futvilei', 'futvolei')
            .replace('handbol', 'handebol')
            .replace('queimada e atletismo', 'queimada, atletismo')
            .replace('voleiboll', 'voleibol')
            .replace('voleiwheeling e moto crossboll', 'wheeling/moto cross')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('esportes_desejados_')

insert_position = df.columns.get_loc('esportes_desejados') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

##### jogos_desejados

In [59]:
df['aux'] = df['jogos_desejados'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace("(toto)", '')
            .replace('pebolin', 'pebolin/toto')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('jogos_desejados_')

insert_position = df.columns.get_loc('jogos_desejados') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

##### lutas_desejadas

In [60]:
df['aux'] = df['lutas_desejadas'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace("kung fuh", 'kung fu')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('lutas_desejadas_')

insert_position = df.columns.get_loc('lutas_desejadas') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)